In [152]:
import tweepy
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt


MY_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAFuXcwEAAAAAnHYkldO1duo%2FfGUE2Ptzhrje4Mc%3DhSmQPaiAH3pjhT1OCuFPVK1z0BbK3iLzqEFSmE59L9Nd56NkSj"

client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

In [153]:
#search_query = "@TheBridge_Tech -in:retweets"
search_query = "@TheBridge_Tech"

In [154]:
#query = "@TheBridge_Tech lang:es -is:retweet"
query = "@TheBridge_Tech"

start_time = "2022-05-16T19:51:00Z"
end_time = "2022-5-23T15:00:00Z"

tweets = client.search_recent_tweets(query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = ["created_at", "text", "source"],
                                     user_fields = ["name", "username", "location", "description"],
                                     max_results = 100,
                                     expansions='author_id'
                                     )

In [155]:
tweets.data

[<Tweet id=1528735286829764608 text='RT @arey: #LastMinute La 2da presentación del #LibroIC en #Madrid será este jueves (18:30) en @TheBridge_Tech acompañados por Tíscar Lara (…'>,
 <Tweet id=1528718102195802114 text='RT @PdelaPublicidad: Los perfiles tecnológicos más demandados en el futuro @TheBridge_Tech https://t.co/j93jBWSLij'>,
 <Tweet id=1528696920729956352 text='Terminando el desafio de tripulaciones en @TheBridge_Tech'>,
 <Tweet id=1528674680965586945 text='RT @nodosenlared: "El libro de la inteligencia colectiva"  https://t.co/tUyIEoFlP9 artículo de @tiscar sobre la presentación que del libro…'>,
 <Tweet id=1528653470085128192 text='RT @nodosenlared: "El libro de la inteligencia colectiva"  https://t.co/tUyIEoFlP9 artículo de @tiscar sobre la presentación que del libro…'>,
 <Tweet id=1528650844094271490 text='"El libro de la inteligencia colectiva"  https://t.co/tUyIEoFlP9 artículo de @tiscar sobre la presentación que del libro de @arey haremos esta semana en @TheBridge_Tech 

In [156]:
tweet_info_ls = []

for tweet, user in zip(tweets.data, tweets.includes['users']):
    tweet_info = {
        'fecha': tweet.created_at,
        'autor_id': tweet.author_id,
        'cuerpo': tweet.text,
        'origen': tweet.source,
        #'RT': tweet.metrics,
        'nombre_autor': user.name,
        'username_autor': user.username,
        'localizacion': user.location,
        'descripcion': user.description
    }
    tweet_info_ls.append(tweet_info)

tweets_df = pd.DataFrame(tweet_info_ls)

tweets_df

,fecha,autor_id,cuerpo,origen,nombre_autor,username_autor,localizacion,descripcion
0,2022-05-23 13:51:09+00:00,1162694149956603904,RT @arey: #LastMinute La 2da presentación del ...,Twitter Web App,The Bridge,TheBridge_Tech,"Paseo de Recoletos, 15, Madrid",Aceleradora de Talento Digital\n\nBootcamps | ...
1,2022-05-23 12:42:52+00:00,287347055,RT @PdelaPublicidad: Los perfiles tecnológicos...,Twitter Web App,Nicole Orchard,NicoleOrchard,Madrid,
2,2022-05-23 11:18:42+00:00,1528649918306557952,Terminando el desafio de tripulaciones en @The...,Twitter Web App,Nacho Esteban,nachooatm82,None,
3,2022-05-23 09:50:20+00:00,135595071,"RT @nodosenlared: ""El libro de la inteligencia...",Twitter for Android,Eva Sánchez,mevasjimenez,None,Long Life learner. Engineer.
4,2022-05-23 08:26:02+00:00,7154412,"RT @nodosenlared: ""El libro de la inteligencia...",Twitter for iPhone,Javier G. Recuenco,Recuenco,"Madrid, Spain",Ex-IT nerd. Complex Problem Solver. CSO @Singu...
5,2022-05-23 08:15:36+00:00,41545659,"""El libro de la inteligencia colectiva"" https...",Buffer,Mario Lopez de Avila,nodosenlared,Wherever my books are,A curious person in a complex world.\nCommitte...
6,2022-05-23 08:01:09+00:00,4628161,Este jueves nos vemos en @TheBridge_Tech para ...,Twitter Web App,Tíscar Lara,tiscar,Madrid,Directora de Transformación Digital de @InstCe...
7,2022-05-23 07:54:21+00:00,1573780194,RT @juanluispavon1: Conoce las 25 ofertas de #...,Twitter for iPhone,Patricia González-Rodríguez #SinCienciaNoHayFu...,pgonzrodriguez,"Seville, Spain",👩🏻‍🔬🧠Neuroscientist. Junior PI @ibis_sevilla @...
8,2022-05-23 07:53:58+00:00,271387101,RT @juanluispavon1: Conoce las 25 ofertas de #...,Twitter for iPhone,Antonio Yélamo Crespillo,Antonioyelamo,None,Periodista. Cadena SER Andalucía
9,2022-05-23 07:38:11+00:00,987388686,Conoce las 25 ofertas de #empleo de la empresa...,Twitter Web App,Juan Luis Pavón,juanluispavon1,"Sevilla, España",Fundador de Sevilla World @WorldSevilla Period...


## MACHINE LEARNING

#### Tengo este segundo codigo tambien pero no me deja instalarme el kernel de la version 3.8 

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import os

dir_path = os.path.dirname(os.path.realpath(file))
os.chdir(dir_path)
query = "@TheBridge_Tech until:2022-05-24 since:2022-01-01"
tweets = []
limit = 4000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():

    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.id, tweet.content, tweet.date, tweet.user.id, tweet.user.displayname, tweet.user.username, tweet.retweetCount, tweet.replyCount, tweet.likeCount, tweet.quoteCount])

df = pd.DataFrame(tweets, columns=['Id_tweet', 'Texto', "Fecha", 'Id_autor', "Nombre_autor", "username_autor", "N_Retweet", "N_Reply", "N_Like", "N_Quote"])
df.to_csv("twitter.csv")
print(df)

### Y un tercero

In [150]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

tweets = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('@TheBridge_Tech').get_items()):
    if i>300:
        break
    tweets.append([tweet.date, tweet.id, tweet.content, tweet.username])

tweets_df1 = pd.DataFrame(tweets, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

tweets_df1.to_csv("tweets.csv")

In [151]:
tweets_df1

,Datetime,Tweet Id,Text,Username
0,2022-05-23 11:18:42+00:00,1528696920729956352,Terminando el desafio de tripulaciones en @The...,nachooatm82
1,2022-05-23 08:15:36+00:00,1528650844094271490,"""El libro de la inteligencia colectiva"" https...",nodosenlared
2,2022-05-23 08:01:09+00:00,1528647207594909696,Este jueves nos vemos en @TheBridge_Tech para ...,tiscar
3,2022-05-23 07:38:11+00:00,1528641426975760389,Conoce las 25 ofertas de #empleo de la empresa...,juanluispavon1
4,2022-05-23 07:25:47+00:00,1528638307516694529,#LastMinute La 2da presentación del #LibroIC e...,arey
...,...,...,...,...
296,2021-08-17 10:05:25+00:00,1427572264418041894,Este otoño @TheBridge_Tech acoge un ciclo de c...,nodosenlared
297,2021-08-13 09:00:00+00:00,1426106250727677952,"Kubernetes, te contamos qué es. 😎\n\n#kubernet...",TheBridge_Tech
298,2021-08-11 09:00:05+00:00,1425381495233990657,Os presentamos nuestro badges de Digital Produ...,TheBridge_Tech
299,2021-08-06 09:00:00+00:00,1423569535072886787,¿Sabes en qué consiste el Research? 😎\n\n#uxui...,TheBridge_Tech
